In [24]:
import fitsio
from astropy.io import fits as pyfits
import numpy as np

In [25]:
# needed for clearing files out of cache
# if you don't care about this, comment out lines below and/or 
# don't run the last cell for timings
import fadvise as fa
from time import sleep

In [26]:
import glob
files = glob.glob('images/*.fits')
print files

['images/scaled15000x15000.fits', 'images/5000x5000.fits', 'images/1000x1000.fits', 'images/15000x15000.fits', 'images/iris.fits', 'images/10000x10000.fits']


In [27]:
def read_test_pyfits(filename):
    '''Use PyFits to open image. Default parameters'''
    with pyfits.open(filename) as hdul:
        data = hdul[0].data
        sum = np.sum(data)

In [28]:
def read_test_fitsio(filename):
    '''Use FitsIO to open image. Default parameters'''
    with fitsio.FITS(filename) as f:
        data = f[0].read()
        sum = np.sum(data)

In [34]:
def read_test_fitsio_buffered(filename, n=1000):
    '''Experimental function for fitsio to open an image in 
    a sequentially buffered fashion. Not used in this notebook, 
    but may be correctly implemented and included in the testing
    in the future.'''
    data =[]

    with fitsio.FITS(filename, iter_row_buffer=n) as f:
        for hdu in f:
            data.append(hdu.read())
        return data

In [30]:
for f in files:
    print f
    %time read_test_fitsio(f)
    fa.dontneed(f)
    print '\n'

images/scaled15000x15000.fits
CPU times: user 1.26 s, sys: 1.11 s, total: 2.38 s
Wall time: 2.87 s


images/5000x5000.fits
CPU times: user 80 ms, sys: 164 ms, total: 244 ms
Wall time: 623 ms


images/1000x1000.fits
CPU times: user 4 ms, sys: 12 ms, total: 16 ms
Wall time: 27.1 ms


images/15000x15000.fits
CPU times: user 768 ms, sys: 1.63 s, total: 2.4 s
Wall time: 5.74 s


images/iris.fits
CPU times: user 1.8 s, sys: 708 ms, total: 2.51 s
Wall time: 2.58 s


images/10000x10000.fits
CPU times: user 356 ms, sys: 592 ms, total: 948 ms
Wall time: 2.49 s




In [31]:
for f in files:
    print f
    %time read_test_pyfits(f)
    fa.dontneed(f)
    print '\n'

images/scaled15000x15000.fits
CPU times: user 1.69 s, sys: 1.04 s, total: 2.74 s
Wall time: 3.52 s


images/5000x5000.fits
CPU times: user 108 ms, sys: 96 ms, total: 204 ms
Wall time: 564 ms


images/1000x1000.fits
CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 22.5 ms


images/15000x15000.fits
CPU times: user 832 ms, sys: 912 ms, total: 1.74 s
Wall time: 5.08 s


images/iris.fits
CPU times: user 2.11 s, sys: 748 ms, total: 2.86 s
Wall time: 2.98 s


images/10000x10000.fits
CPU times: user 400 ms, sys: 392 ms, total: 792 ms
Wall time: 2.24 s




In [32]:
# may not be a useful test, since the first call caches the image, 
# making the second call cached and faster
# we'll try to advise the cache to remove the image and add a delay

for f in files:
    
    print '```' + f
    print '-' + 'fitsio'
    fa.dontneed(f)
    %time read_test_fitsio(f)
    fa.dontneed(f)
    sleep(5)
    print '-' + 'pyfits'
    %time read_test_pyfits(f)
    fa.dontneed(f)
    print '\n'

***images/scaled15000x15000.fits
-fitsio
CPU times: user 1.24 s, sys: 1.19 s, total: 2.43 s
Wall time: 2.79 s
-pyfits
CPU times: user 1.69 s, sys: 1.06 s, total: 2.74 s
Wall time: 3.48 s


***images/5000x5000.fits
-fitsio
CPU times: user 100 ms, sys: 152 ms, total: 252 ms
Wall time: 621 ms
-pyfits
CPU times: user 108 ms, sys: 100 ms, total: 208 ms
Wall time: 580 ms


***images/1000x1000.fits
-fitsio
CPU times: user 4 ms, sys: 8 ms, total: 12 ms
Wall time: 26.9 ms
-pyfits
CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 22.3 ms


***images/15000x15000.fits
-fitsio
CPU times: user 684 ms, sys: 1.78 s, total: 2.47 s
Wall time: 5.73 s
-pyfits
CPU times: user 948 ms, sys: 832 ms, total: 1.78 s
Wall time: 5.04 s


***images/iris.fits
-fitsio
CPU times: user 1.88 s, sys: 804 ms, total: 2.68 s
Wall time: 2.72 s
-pyfits
CPU times: user 2.08 s, sys: 752 ms, total: 2.83 s
Wall time: 2.96 s


***images/10000x10000.fits
-fitsio
CPU times: user 344 ms, sys: 592 ms, total: 936 ms
Wall time: 2

## Conclusions

There are no discernable differences between loading a image with `fitsio` and `pyfits`. There is a slight time improvement with larger files on `pyfits`, implying that `pyfits` might be better at loading big images. The maximum image size tested here was ~800 MB, so to discern any signficant time improvement, we might need to test with files around 2-3 GB. 